# Scalar advection-diffusion using a parallel-in-time method

Now that we are familiar with scalar advection-diffusion, we will solve it using the parallel-in-time method ParaDiag.
We write the scalar advection-diffusion equations as:
$$
\partial_{t}q + u\partial_{x}q - \nu\partial^{2}_{xx}q = \partial_{t}q + \textbf{K}q = 0
$$
with $\textbf{K}$ containing the spatial terms.

The implicit theta method discretisation then looks like:

$$
\frac{q^{n+1}-q^{n}}{\Delta t} + \theta \textbf{K}q^{n+1} + (1-\theta)\textbf{K}q^{n} = 0
$$

We can now construct the all-at-once system which couples all timesteps in the time-series together into a single problem. For 4 timesteps this looks like:

$$
\left(
\frac{1}{\Delta t}
\begin{pmatrix}
 1 &  0 &  0 & 0 \\
-1 &  1 &  0 & 0 \\
 0 & -1 &  1 & 0 \\
 0 &  0 & -1 & 1 \\
\end{pmatrix}
\otimes\textbf{M}
+
\begin{pmatrix}
\theta     & 0          & 0          & 0 \\
(1-\theta) & \theta     & 0          & 0 \\
0          & (1-\theta) & \theta     & 0 \\
0          & 0          & (1-\theta) & \theta \\
\end{pmatrix}
\otimes\textbf{K}
\right)
\begin{pmatrix}
q^{1} \\ q^{2} \\ q^{3} \\ q^{4} \\
\end{pmatrix}
=
\textbf{b}
$$

where $\otimes$ is the [kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) and $\textbf{M}=\textbf{I}_{x}$ is the mass matrix, again the identity matrix of size $n_{x}\times n_{x}$.
As with Dalhquist's equation, the left hand side can be written using the two Toeplitz matrices $\textbf{B}_{1}$ and $\textbf{B}_{2}$ which define the implicit theta method:

$$
\textbf{A}\textbf{q} = \left(\frac{1}{\Delta t}\textbf{B}_{1}\otimes\textbf{M} + \textbf{B}_{2}\otimes\textbf{K}\right)\textbf{q}
$$

Where $\textbf{A}$ is the all-at-once Jacobian, and $\textbf{q}$ is the vector of the timeseries $q^{n}$.

The right hand side vector $\textbf{b}$ contains the initial condition:

$$
\begin{pmatrix}
q^{0}/\Delta t - (1-\theta)\textbf{K}q^{0} \\ 0 \\ 0 \\ 0
\end{pmatrix}
$$

## Implementing the all-at-once system

We start as for the serial method by defining the problem parameters.

In [ ]:
from math import pi
import numpy as np

nt = 128
nx = 128

lx = 2*pi
dx = lx/nx

theta = 0.5

# velocity, CFL, and reynolds number
u = 1
re = 500
cfl = 0.8

# viscosity and timestep
nu = lx*u/re
dt = cfl*dx/u

# advective and diffusive Courant numbers

cfl_u = cfl
cfl_v = nu*dt/dx**2

print(f"{nu = }, {dt = }, {cfl_v = }, {cfl_u = }")

# Spatial domain
mesh = np.linspace(start=-lx/2, stop=lx/2, num=nx, endpoint=False)

Because we will be using GMRES to solve the all-at-once system, we will need to implement a `LinearOperator` to carry out a matrix-vector product with the all-at-once Jacobian $\textbf{A}$. This will be built from The same sparse matrices we built for the serial-in-time method previously. There are two unique blocks, which are repeated on each (block) row.
We can see this by rearranging the theta method discretisation to group the terms involving each timestep together:

$$
\left(\frac{-\textbf{M}}{\Delta t} + (1-\theta)\textbf{K}\right)q^{n} +
\left(\frac{\textbf{M}}{\Delta t} + \theta \textbf{K}\right)q^{n+1}
= \textbf{A}_{0}q^{n} + \textbf{A}_{1}q^{n+1} = 0
$$

We will use the same finite difference discretisation for the spatial terms. Here are the same convenience functions for creating the stencil matrices:


In [ ]:
from scipy import sparse
from scipy import linalg
from scipy.sparse import linalg as spla

# Finite difference spatial discretisations                                                                                                                                                   
def gradient_stencil(grad, order):                                                                                                                                                            
    '''                                                                                                                                                                                       
    Return the centred stencil for the `grad`-th gradient                                                                                                                                     
    of order of accuracy `order`                                                                                                                                                              
    '''                                                                                                                                                                                       
    return {                                                                                                                                                                                  
        1: {  # first gradient                                                                                                                                                                
            2: np.array([-1/2, 0, 1/2]),                                                                                                                                                      
            4: np.array([1/12, -2/3, 0, 2/3, -1/12]),                                                                                                                                         
            6: np.array([-1/60, 3/20, -3/4, 0, 3/4, -3/20, 1/60])                                                                                                                             
        },                                                                                                                                                                                    
        2: {  # second gradient                                                                                                                                                               
            2: np.array([1, -2, 1]),                                                                                                                                                          
            4: np.array([-1/12, 4/3, -5/2, 4/3, -1/12]),                                                                                                                                      
            6: np.array([1/90, -3/20, 3/2, -49/18, 3/2, -3/20, 1/90])                                                                                                                         
        },                                                                                                                                                                                    
        4: {  # fourth gradient                                                                                                                                                               
            2: np.array([1,  -4, 6, -4, 1]),                                                                                                                                                  
            4: np.array([-1/6, 2, -13/2, 28/3, -13/2, 2, -1/6]),                                                                                                                              
            6: np.array([7/240, -2/5, 169/60, -122/15, 91/8, -122/15, 169/60, -2/5, 7/240])  # noqa: E501                                                                                     
        }                                                                                                                                                                                     
    }[grad][order]                                                                                                                                                                            
                                                                                                                                                                                              
                                                                                                                                                                                              
def sparse_circulant(stencil, n):                                                                                                                                                             
    '''                                                                                                                                                                                       
    Return sparse scipy matrix from finite difference                                                                                                                                         
    stencil on a periodic grid of size n.                                                                                                                                                     
    '''                                                                                                                                                                                       
    if len(stencil) == 1:                                                                                                                                                                     
        return sparse.spdiags([stencil[0]*np.ones(n)], 0)                                                                                                                                     
                                                                                                                                                                                              
    # extend stencil to include periodic overlaps                                                                                                                                             
    ns = len(stencil)                                                                                                                                                                         
    noff = (ns-1)//2                                                                                                                                                                          
    pstencil = np.zeros(ns+2*noff)                                                                                                                                                            
                                                                                                                                                                                              
    pstencil[noff:-noff] = stencil                                                                                                                                                            
    pstencil[:noff] = stencil[noff+1:]                                                                                                                                                        
    pstencil[-noff:] = stencil[:noff]                                                                                                                                                         
                                                                                                                                                                                              
    # constant diagonals of stencil entries                                                                                                                                                   
    pdiags = np.tile(pstencil[:, np.newaxis], n)                                                                                                                                              
                                                                                                                                                                                              
    # offsets for inner domain and periodic overlaps                                                                                                                                          
    offsets = np.zeros_like(pstencil, dtype=int)                                                                                                                                              
                                                                                                                                                                                              
    offsets[:noff] = [-n+1+i for i in range(noff)]                                                                                                                                            
    offsets[noff:-noff] = [-noff+i for i in range(2*noff+1)]                                                                                                                                  
    offsets[-noff:] = [n-noff+i for i in range(noff)]                                                                                                                                         
                                                                                                                                                                                              
    return sparse.spdiags(pdiags, offsets)

We will need to create blocks with different coefficients, so we make a function to generate these:

In [ ]:
# Mass matrix                                                                                                                                                                                 
M = sparse_circulant([1], nx)                                                                                                                                                                 
                                                                                                                                                                                              
# Advection matrix                                                                                                                                                                            
D = sparse_circulant(gradient_stencil(1, order=2), nx)                                                                                                                                        
                                                                                                                                                                                              
# Diffusion matrix                                                                                                                                                                            
L = sparse_circulant(gradient_stencil(2, order=2), nx)

# Spatial terms                                                                                                                                                                               
K = (u/dx)*D - (nu/dx**2)*L

# Generate block matrices for different coefficients                                                                                                                                          
def block_matrix(l1, l2):                                                                                                                                                                     
    mat = l1*M + l2*K                                                                                                                                                                         
    mat.solve = spla.factorized(mat.tocsc())                                                                                                                                                  
    return mat

To build the all-at-once Jacobian $\textbf{A}$ we need the timestepping matrices $\textbf{B}_{1,2}$ and the blocks $\textbf{A}_{0,1}$. You can create these based on the serial-in-time code.

Once these four matrices have been created the full all-at-once Jacobian $\textbf{A}$ can be built. Once $\textbf{A}$ is constructed, you should use `spla.aslinearoperator` to convert it to a `LinearOperator` which can be used with the `gmres` method.

_`scipy.sparse.kron` will be useful for implementing this `LinearOperator`._

In [ ]:
# Define the Toeplitz columns/rows
b1col = np.zeros(nt)
b1col[0] = 1/dt
b1col[1] = -1/dt

b1row = np.zeros_like(b1col)
b1row[0] = b1col[0]

b2col = np.zeros(nt)
b2col[0] = theta
b2col[1] = 1-theta

b2row = np.zeros_like(b2col)
b2row[0] = b2col[0]

# build the full B1, B2 matrices

# Build the A0 and A1 matrices

# Now build the full Jacobian A

Now we have our numerical scheme set up, time for the initial conditions and the right hand side of the all-at-once system. We use the same initial condition as we did when solving serially in time. Fill in the code to calculate the right hand side from the initial condition.

In [ ]:
from math import pi

qinit = np.zeros_like(mesh)
qinit[:] = np.cos(mesh/2)**4

# set up timeseries                                                                                                                                                                           
q = np.zeros(nt*nx)
q = q.reshape((nt, nx))

# initial guess is constant solution                                                                                                                                                          
q[:] = qinit[np.newaxis, :]
q = q.reshape(nx*nt)

# calculate the right hand side with the contribution of the initial condition here
rhs = np.zeros_like(q)                                                                                                                                                                        

import matplotlib.pyplot as plt
plt.plot(mesh, qinit)
plt.show()

We will first try to solve the all-at-once system with brute force by unpreconditioned GMRES. Let's see how that goes.

In [ ]:
niterations=0
def gmres_callback(y):
    global niterations
    print(f"niterations: {str(niterations).rjust(3,' ')}  |  residual:  {y}")
    niterations += 1
    return

q, exit_code = spla.gmres(A, rhs,
                          restart=100,
                          callback=gmres_callback,
                          callback_type='pr_norm')

In [ ]:
print(f"gmres exit code: {exit_code}")                                                                                                                 
print(f"gmres iterations: {niterations}")
print(f"residual: {linalg.norm(rhs-A.matvec(q))}")

Just as with Dahlquist's ODE we see that the unpreconditioned all-at-once system takes many GMRES iterations to solve (this isn't unique to all-at-once matrices, it is very common for Krylov methods to perform poorly without appropriate preconditioning). We will again try to reduce the number of iterations needed for the GMRES solve using a circulant preconditioner. This time the preconditioner will be block circulant, with blocks of size $n_{x}\times n_{x}$ to match the blocks of the all-at-once Jacobian. We will go straight to considering the $\alpha$-circulant preconditioner.

The $\alpha$-circulant preconditioner has the following form:

$$
\textbf{P} =
\frac{1}{\Delta t}
\begin{pmatrix}
 1 &  0 &  0 & -\alpha \\
-1 &  1 &  0 & 0 \\
 0 & -1 &  1 & 0 \\
 0 &  0 & -1 & 1 \\
\end{pmatrix}
\otimes\textbf{M}
+
\begin{pmatrix}
\theta     & 0          & 0          & \alpha(1-\theta) \\
(1-\theta) & \theta     & 0          & 0 \\
0          & (1-\theta) & \theta     & 0 \\
0          & 0          & (1-\theta) & \theta \\
\end{pmatrix}
\otimes\textbf{K}
$$

which can be written more compactly as:

$$
\textbf{P} = \textbf{C}^{(\alpha)}_{1}\otimes\textbf{M} + \textbf{C}^{(\alpha)}_{2}\otimes\textbf{K}
$$

It is a property of the [Kronecker product $\otimes$](https://en.wikipedia.org/wiki/Kronecker_product) that if $\textbf{U}$, $\textbf{V}$ are $m\times m$ matrices and $\textbf{X}$, $\textbf{Y}$ are $n\times n$ matrices, then:

$$
\left(\textbf{UV}\right)\otimes\left(\textbf{XY}\right)
=
\left(\textbf{U} \otimes \textbf{X}\right)\left(\textbf{V} \otimes \textbf{Y}\right)
$$

This property can be used to show that if $\textbf{C}$ admits the factorisation:

$$
\textbf{C} = \textbf{V}\textbf{D}\textbf{V}^{-1}
$$

then the Kronecker product with this matrix admits the factorisation:

$$
\textbf{C}\otimes\textbf{K} =
\left(\textbf{V}\otimes\textbf{I}\right)
\left(\textbf{D}\otimes\textbf{K}\right)
\left(\textbf{V}^{-1}\otimes\textbf{I}\right)
$$

where $\textbf{I}$ is the identity matrix with the same size as $\textbf{K}$. Remember that the two circulant matrices $\textbf{C}^{(\alpha)}_{1,2}$ in our preconditioner can be simultaneously diagonalised. Combining this with the Kronecker product property above, we can see that our preconditioner can be factorised as:

$$
\textbf{P} =
\left(\textbf{V}\otimes\textbf{I}_{x}\right)
\left(\textbf{D}_{1}\otimes\textbf{M} + \textbf{D}_{2}\otimes\textbf{K}\right)
\left(\textbf{V}^{-1}\otimes\textbf{I}_{x}\right)
$$

There are two important points to note about this factorisation.
1. The action of $\left(\textbf{V}^{-1}\otimes\textbf{I}_{x}\right)$ on an all-at-once vector (i.e. a vector of size $n_{t}n_{x}$ arranged as the concatenation of $n_{t}$ vectors of size $n_{x}$) is equivalent to multiplying the time series at each grid point by the matrix $\textbf{V}^{-1}$. For $\alpha=1$ where $\textbf{V}^{-1}$ is the DFT matrix, this is equivalent to applying a Fourier transform in time at each grid point.
2. Because $\textbf{D}_{i}=\texttt{diag}(\lambda_{i,n}),\, i=1,2$ are diagonal matrices of the eigenvalues $\lambda$ of $\textbf{C}^{(\alpha)}_{1,2}$, the matrix in the middle set of brackets is block diagonal, meaning that each of these blocks can be solved _independently_.

We can now write the three steps for efficiently solving systems of the form $\textbf{P}\textbf{x}=\textbf{b}$

1. Apply the matrix $\textbf{V}^{-1}$ to the timeseries at each grid point:
    $\textbf{y}_{1}=(\textbf{V}^{-1}\otimes\textbf{I}_{x})\textbf{b}$
2. Solve the block diagonal matrix by splitting the vector $\textbf{y}$ into $n_{t}$ vectors of size $n_{x}$ and inverting each block:
    $(\lambda_{1,n}\textbf{I}+\lambda_{2,n}\textbf{K})\textbf{y}_{2,n}=\textbf{y}_{1,n}\, \forall n\in[1,n_{t}]$
3. Apply the matrix $\textbf{V}$ to the timeseries at each grid point:
    $\textbf{x}=(\textbf{V}\otimes\textbf{I}_{x})\textbf{y}_{2}$

We are almost ready to implement this preconditioner, but before we do here is a reminder of the expressions for the eigenvectors and eigenvalues of the circulant matrices:
$$
\textbf{C}^{(\alpha)} = \textbf{V}\textbf{D}\textbf{V}^{-1}
$$
where:
$$
\textbf{V}^{-1} = \textbf{F}\Gamma_{\alpha},
\quad
\textbf{D} = \textbf{F}\Gamma_{\alpha}\textbf{C}[:,0]
$$
and $\Gamma_{\alpha}$ is the diagonal matrix:
$$
\Gamma_{\alpha} = \text{diag}\left(\alpha^{\frac{k-1}{N_{t}}}\right)\; \forall k \in [1, N_{t}]
$$


Now finally it is time to implement the `LinearOperator` for our preconditioner by completing the class skeleton below by finishing the `__init__` method and implementing the `_matvec` method.

_Our problem is real-valued, so remember to return only the real part of the solution vector from the `_matvec` method._

_Every step of applying the preconditioner can be formulated as either an operation on the timeseries of length $n_{t}$ at each of the $n_{x}$ mesh points, or an operation on $n_{t}$ vectors of size $n_{x}$. The `.reshape` method of NumPy arrays can make this simpler to code._

_It might be easier to implement first for the $\alpha=1$ case only and finish the notebook, then return to implement for $\alpha\neq1$._

In [ ]:
from scipy.fft import fft, ifft

class BlockCirculantLinearOperator(spla.LinearOperator):
    def __init__(self, b1col, b2col, block_matrix, nx, alpha=1):
        """
        :arg b1col: the first column of B1
        :arg b2col: the first column of B2
        :arg block_matrix: a generating function for the block matrices, called as block_matrix(l1, l2)
        :arg nx: the number of spatial mesh points
        :arg alpha: the circulant parameter
        """
        self.nt = len(b1col)
        self.nx = nx
        self.dim = self.nt*self.nx
        self.shape = tuple((self.dim, self.dim))
        self.dtype = b1col.dtype

        # finish setting up the preconditioner here

    def _matvec(self, v):
        # implement this method

Now to test the preconditioner and see how much it improves the GMRES convergence

In [ ]:
alpha = 0.01
P = BlockCirculantLinearOperator(b1col, b2col, block_matrix, nx, alpha)

niterations=0
q, exit_code = spla.gmres(A, rhs, M=P,
                          callback=gmres_callback,
                          callback_type='pr_norm')

print(f"gmres exit code: {exit_code}")                                                                                                                 
print(f"gmres iterations: {niterations}")
print(f"residual: {linalg.norm(rhs-A.matvec(q))}")

That improves the convergence quite a lot! The circulant preconditioner is effective for this PDE as well as Dahlquists ODE.

Let's plot the solution to check it looks as we expect.

In [ ]:
q = q.reshape((nt,nx))
nplot=16
plt.plot(mesh, qinit, label='i')
for i in range(nplot, nt, nplot):
    plt.plot(mesh, q[i], label=str(i))
plt.legend(loc='center left')
plt.grid()
plt.show()

Now you can explore the behaviour of the preconditioned system. Some suggestions for questions to ask are:
- How does varying $\alpha$ affect the residuals?
- What happens when $\alpha$ approaches machine zero?
- What does the (unconverged) solution vector look like after just one iteration? After two iterations? Three?
- What happens if you change the problem parameters, e.g. `nt`, `nx`, `dt`, `re`? Does the value of $\alpha$ affect the parameter dependence?

Remember that you can change to tolerance of the GMRES solve using the `rtol` and `atol` kwargs.
